In [3]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\ahunt\anaconda3\envs\pythondata\lib\site-packages (0.0)


In [4]:
# install joblib. This will be used to save your model. 
!pip install joblib

In [5]:
import pandas as pd

## Read the CSV and Perform Basic Data Cleaning ##

In [6]:
df = pd.read_csv("exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Display dataframe
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


## Select the Features and Label Values ##

In [7]:
# Set features as X value and label as y value
# Use koi_disposition for the y_values
X = df.drop(columns=['koi_disposition'])
y = df['koi_disposition']

## Create a Train/Test Split ##

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## Pre-Processing ##

In [9]:
#Scale the data using the MinMaxScaler and perform some feature selection
from sklearn.preprocessing import MinMaxScaler

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

## Create the Model ##

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
model = RandomForestClassifier(random_state = 1)

# Fit (train) or model using the training data
model = model.fit(X_train_scaled, y_train)

In [11]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8941647597254004


In [12]:
# print prediction results
from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.74      0.78       422
     CONFIRMED       0.80      0.84      0.82       450
FALSE POSITIVE       0.97      1.00      0.98       876

      accuracy                           0.89      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



## Identifying Insignificant Variables ##

In [13]:
# Sort the features by their importance
features_imp = pd.DataFrame({
    'features': X.columns,
    'probability': model.feature_importances_
})

features_imp = features_imp.sort_values('probability', ascending=False)

features_imp.head()

,features,probability
2,koi_fpflag_co,0.107540
0,koi_fpflag_nt,0.104147
1,koi_fpflag_ss,0.073996
26,koi_model_snr,0.057301
19,koi_prad,0.052542


In [14]:
# Locate features with probability > 0.01
selected_features_df = features_imp.loc[features_imp['probability'] > 0.01]

# Get features based on above condition
selected_features = list(selected_features_df['features'])

selected_features

['koi_fpflag_co',
 'koi_fpflag_nt',
 'koi_fpflag_ss',
 'koi_model_snr',
 'koi_prad',
 'koi_duration_err2',
 'koi_fpflag_ec',
 'koi_prad_err2',
 'koi_steff_err1',
 'koi_duration_err1',
 'koi_duration',
 'koi_period',
 'koi_prad_err1',
 'koi_time0bk_err1',
 'koi_depth',
 'koi_steff_err2',
 'koi_time0bk_err2',
 'koi_period_err2',
 'koi_insol_err1',
 'koi_period_err1',
 'koi_impact',
 'koi_teq',
 'koi_time0bk',
 'koi_depth_err1',
 'koi_depth_err2',
 'dec',
 'koi_insol',
 'ra',
 'koi_impact_err1',
 'koi_insol_err2',
 'koi_impact_err2',
 'koi_slogg']

## Train the Model ##

In [15]:
# Set features as X value and label as y value
X = df[selected_features]
y = df['koi_disposition']

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [17]:
from sklearn.preprocessing import MinMaxScaler

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

In [18]:
# Create a random forest classifier
model = RandomForestClassifier(random_state = 1)

# Fit (train) or model using the training data
model = model.fit(X_train_scaled, y_train)

In [19]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.8913043478260869


In [20]:
predictions = model.predict(X_test_scaled)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.73      0.78       422
     CONFIRMED       0.79      0.84      0.81       450
FALSE POSITIVE       0.97      1.00      0.98       876

      accuracy                           0.89      1748
     macro avg       0.86      0.85      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



## Hyperparameter Tuning ##

In [21]:
#Use GridSearchCV to tune the model's parameters
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import FitFailedWarning
import warnings
warnings.simplefilter('always', FitFailedWarning)


# Defining parameter range 
param_grid = {
    'max_depth': [5, 8, 15, 25, 30],
    'min_samples_leaf': [1, 2, 5, 10] ,
    'min_samples_split': [2, 5, 10, 15, 100],
    'n_estimators': [100, 300, 500, 800, 1200]
}
  
grid = GridSearchCV(model, param_grid, cv = 3, n_jobs = -1, verbose = 1)

In [22]:
# Fit the model using the grid search estimator.
with warnings.catch_warnings(record=True) as w:
    try:
        grid.fit(X_train_scaled, y_train)
    except ValueError:
         pass

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


In [23]:
print(f'Best Parameter: {grid.best_params_}')
print(f'Best Model Score: {grid.best_score_}')

Best Parameter: {'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}
Best Model Score: 0.8983419107485995


In [24]:
predictions = grid.predict(X_test_scaled)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.83      0.75      0.79       422
     CONFIRMED       0.80      0.84      0.82       450
FALSE POSITIVE       0.97      1.00      0.98       876

      accuracy                           0.89      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748



## Save the Model ##

In [25]:
import joblib
from joblib import dump, load

# save model
filename = 'RF_Model.sav'
joblib.dump(grid, filename)

['RF_Model.sav']